In [63]:
import re
import pandas as pd

In [64]:
SEED = 42

In [65]:
prompt_dataset = pd.read_csv('api_dataset.csv')

In [66]:
prompt_dataset.head()

,Unnamed: 0,PROMPT,SCENARIO
0,0,"Giant Biker Gladiator, fullbody; by Peter Mohr...",아이: 저는 거대한 바이커 격투사 그림을 그리고 싶어요. 그리고 그림에는 꼭 Pet...
1,0,crossing the Shadow fold into the pure darknes...,아이: 저번에 본 것처럼 Shadow fold를 그리고 Volcra creature...
2,0,"fire dragon, sitting in dragon nest, dragon, 4...",아이: 용 둥지에 앉은 불타는 용 그림을 그려봐. 그리고 용의 비늘 디테일도 넣어줘...
3,0,"Woody McBride techno rave poster, big yellow s...",아이: 우디 맥브라이드의 테크노 레이브 포스터를 만들어봐. 큰 노란색 스마일리도 추...
4,0,savage barbarian victorian shaman warrior cybo...,아이: 저에게 savage barbarian victorian shaman warr...


In [67]:
def clean_text(dialogue):
    
    dialogue_list = []
    
    for s in dialogue.split('\n'):
        if s != '':
            s = s.strip()
            if s.startswith('\"') and s.endswith('\"'):
                s = s[1:-1]
            dialogue_list.append(s)
    
    return dialogue_list

In [68]:
def contains_english(text):
    # Define the regular expression pattern to match English characters
    pattern = r'[a-zA-Z]'
    
    # Use the search method to check if the pattern is found in the text
    if re.search(pattern, text):
        return True
    else:
        return False

In [69]:
prompt_dataset['SCENARIO'] = prompt_dataset['SCENARIO'].apply(lambda x: clean_text(x))

In [70]:
len(prompt_dataset)

1917

In [71]:
drop_list = []

# 총 1917개
for i, row in prompt_dataset.iterrows():
    if len(row['SCENARIO']) <= 4: # 대화 개수가 4개보다 적은 경우: 324개
        drop_list.append(i)
    
    elif len(row['PROMPT']) < 10: # prompt 길이가 10보다 작은 경우: 21개
        drop_list.append(i)
   
    else:
        for s in row['SCENARIO']: 
            if '아이' in s and 'Model' in s: # 대화가 잘못 섞여들어간 경우: 38개
                drop_list.append(i)
                break
        
            elif '아이' in s and contains_english(s): # 아이의 대화에 영어가 포함된 경우: 613개
                drop_list.append(i)
                break

In [72]:
prompt_dataset.drop(drop_list, inplace=True)

In [73]:
len(prompt_dataset)

921

In [74]:
prompt_dataset.head()

,Unnamed: 0,PROMPT,SCENARIO
6,0,"highly detailed photograph, 4k, of a lobby : :...","[아이: 저는 요즘 브루탈리스트 건축물에 관심이 생겨서, 로비를 만들어봐주세요., ..."
11,0,"child's drawing of the end of the world, crayo...","[아이: 하얀 종이에 세상의 끝을 그려봐. 크레용으로 그리면 좋겠어., Model:..."
14,0,"small texture, black and white, used look, abs...","[아이: 그림을 그려주세요., Model: What kind of picture w..."
15,0,"Victorian era photographs, spirit photography,...",[아이: 요즘 유행하는 영화처럼 귀신들이 나오는 사진을 만들어봐. 그리고 그 사진에...
20,0,Divine syncronicities between Christianity and...,[아이: 성경과 이슬람교의 신비한 유사점을 그림으로 나타내봐. 그리고 그림에는 조상...


In [75]:
from datasets import Dataset

In [76]:
prompt_dataset = Dataset.from_pandas(prompt_dataset)
# Split the dataset into train and test sets (90% train, 10% test)
prompt_dataset = prompt_dataset.train_test_split(test_size=0.1, seed=SEED)

# Split the train set further into train and validation sets (90% train, 10% validation)
prompt_dataset['train'] = prompt_dataset['train'].train_test_split(test_size=0.1)

# Access the resulting datasets
train_dataset = prompt_dataset['train']['train'].to_pandas()
valid_dataset = prompt_dataset['train']['test'].to_pandas()
test_dataset = prompt_dataset['test'].to_pandas()

In [77]:
train_dataset

,Unnamed: 0,PROMPT,SCENARIO,__index_level_0__
0,0,"door to the stars in the desert, epic, pulp ar...",[아이: 사막에서 별문을 연상시키는 문을 그려봐. 그리고 판타지 잡지 같은 느낌도 ...,790
1,0,glowing brutalist corporate office sleek shiny...,[아이: 사무실 같은 건물을 그려봐. 그리고 건물 주변에 사람들이 많이 있으면 좋겠...,1771
2,0,"a mystic Log cabin, dusk, foggy, mysterious, c...",[아이: 미스틱한 로그 캐빈 같은 거 그려줘. 그리고 땅에 안개가 자욱하게 깔리는 ...,813
3,0,"Maasai warriors, Kenya, trending on artstation...",[아이: 케냐 마사이 전사들을 그린 그림을 만들어줘. 그리고 아트스테이션에서 인기있...,91
4,0,view of a fast food stand that sells squid and...,"[아이: 빨간색 램엔과 오징어를 파는 패스트푸드 가게를 그려줘., Model: vi...",1066
...,...,...,...,...
740,0,"inside a space station, syd mead style, hyperr...",[아이: 우주선 안에서 시드 미드 스타일의 그림을 그려봐. 그리고 너무 어두운 느낌...,1167
741,0,"the city of time, space, and entropy, vast and...",[아이: 저는 미래 도시 같은 그림을 그려봐요. 그리고 빛이 반짝이는 느낌도 있으면...,1701
742,0,beautiful girl with short wavy black hair who ...,[아이: 진짜 멋진 여자 화보 사진을 그려줘. 그리고 머리는 검은머리 파마로 해줘....,608
743,0,"Hokkaido, high detail, 8k, by Ashley Wood --ar...","[아이: 눈사람 그려줘. 눈이 많이 와서 눈사람도 눈이 많이 덮였으면 좋겠어., M...",196


In [78]:
def make_accumulate_dataset(df, file_name):
    accumulate_prompts = []
    accumulate_inputs = []
    outputs = []
    accumulate_scenarios = []

    for i, row in df.iterrows():
        j = 0
        
        while 2 * j + 1 < len(row['SCENARIO']):
            accumulate_prompts.append(row['PROMPT'])
            accumulate_inputs.append(row['SCENARIO'][:2*j+1])
            outputs.append(row['SCENARIO'][2*j+1])
            accumulate_scenarios.append('\n'.join(row['SCENARIO'][:2*j+2]))
            
            j += 1

    new_dataset = pd.DataFrame()
    new_dataset['prompt'] = accumulate_prompts
    new_dataset['input'] = accumulate_inputs
    new_dataset['output'] = accumulate_inputs
    new_dataset['scenario'] = accumulate_scenarios
    
    new_dataset.to_csv(file_name, index=False)

In [79]:
make_accumulate_dataset(train_dataset, 'train.csv')
make_accumulate_dataset(valid_dataset, 'valid.csv')
make_accumulate_dataset(test_dataset, 'test.csv')